In [2]:
import geopandas as gpd
from shapely.geometry import box
# 加载矢量数据（多个 Polygon）
shp_path = r'D:\file\data extent\extent.shp'
gdf = gpd.read_file(shp_path)

gdf_bounds = gdf.geometry.bounds
print(gdf_bounds)
#gdf['bbox_geom'] = gdf_bounds.apply(lambda row: box(row.minx, row.miny, row.maxx, row.maxy), axis=1)


Copernicus_DSM_10_N45_00_E004_00_DEM.tif, Copernicus_DSM_10_N45_00_E005_00_DEM.tif, Copernicus_DSM_10_N46_00_E004_00_DEM.tif, Copernicus_DSM_10_N46_00_E005_00_DEM.tif
Copernicus_DSM_10_N45_00_E015_00_DEM.tif, Copernicus_DSM_10_N45_00_E016_00_DEM.tif
Copernicus_DSM_10_N44_00_E033_00_DEM.tif, Copernicus_DSM_10_N44_00_E034_00_DEM.tif, Copernicus_DSM_10_N45_00_E033_00_DEM.tif, Copernicus_DSM_10_N45_00_E034_00_DEM.tif
Copernicus_DSM_10_N33_00_W078_00_DEM.tif, Copernicus_DSM_10_N33_00_W079_00_DEM.tif
Copernicus_DSM_10_N33_00_E130_00_DEM.tif, Copernicus_DSM_10_N33_00_E131_00_DEM.tif, Copernicus_DSM_10_N34_00_E130_00_DEM.tif, Copernicus_DSM_10_N34_00_E131_00_DEM.tif
Copernicus_DSM_10_N33_00_E132_00_DEM.tif
Copernicus_DSM_10_N33_00_W099_00_DEM.tif, Copernicus_DSM_10_N34_00_W099_00_DEM.tif
Copernicus_DSM_10_N33_00_W079_00_DEM.tif, Copernicus_DSM_10_N33_00_W080_00_DEM.tif, Copernicus_DSM_10_N34_00_W079_00_DEM.tif, Copernicus_DSM_10_N34_00_W080_00_DEM.tif
Copernicus_DSM_10_N33_00_E130_00_DEM.tif
C